In [119]:
import pandas as pd
import json
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account


In [120]:
#@title Service Account Json

# Carregue o arquivo JSON com suas credenciais do Google Cloud
credentials = service_account.Credentials.from_service_account_info(
    json.loads(open("projeto-noticias-394817-324ae2c7db36.json", "r").read())
)

In [103]:
#@title Tratamento do arquivo Json
# Carregue o arquivo JSON em uma estrutura de dados Python
with open('notices.json', 'r') as json_file:
    data = json.load(json_file)

In [104]:
df = pd.DataFrame(data)

In [121]:
def limpar_string(valor_coluna):
    """
    Função realiza o replace nos valores "n\" que vieram junto com a raspagem
    Os valores de 'Date' também vieram com o horário de publicação, a função
    realiza um split nessas informações. Ex: '19.jul.2023 às 19h46'
    """
    if isinstance(valor_coluna, str):
        if 'às' in valor_coluna:
            parte_data, parte_hora = valor_coluna.split('às')
            parte_data = parte_data.strip()
            # Reconstrói a string sem a parte da hora
            string_limpa = f"{parte_data}"
        else:
            # Se não houver 'parte da hora', mantém a string original
            string_limpa = valor_coluna
        return string_limpa.replace('\n', '').strip()
    else:
        return valor_coluna


In [122]:
# Aplica a função limpar_string nas colunas 'title', 'date' e 'newspaper_section'
df['title'] = df['title'].apply(limpar_string)
df['date'] = df['date'].apply(limpar_string)
df['newspaper_section'] = df['newspaper_section'].apply(limpar_string)

In [123]:
def converter_format_date(date_str):
    """
    Função altera valores de meses abreviados na coluna 'Date' para
    os números respectivos de cada mês e também converte o formato.
    """
    try:
        meses_str = [
            'jan', 'fev', 'mar', 'abr', 'mai', 'jun',
            'jul', 'ago', 'set', 'out', 'nov', 'dez'
        ]

        date_obj = datetime.strptime(date_str, '%d.%b.%Y')
        meses_str = date_str.split('.')[1].lower()
        numero_mes = meses_str.index(meses_str)
        date = date_obj.replace(month=numero_mes).strftime('%Y-%m-%d')
        return date
    except:
        return date_str


In [124]:
# Aplica a função de conversão na coluna 'date'
df['date'] = df['date'].apply(converter_format_date)

In [ ]:
df.head()

In [126]:
#@title Parte do código onde os valores são lançados no Big Query

# Substitua project_id pelo ID do seu projeto no Google Cloud
project_id = 'projeto-noticias-394817'


In [113]:
# Substitua dataset_id pelo ID do conjunto de dados que você deseja usar
dataset_id = 'noticias'

In [114]:
# Substitua table_id pelo ID da tabela que você deseja usar
table_id = 'folha'

In [115]:
# Envia o DataFrame para o BigQuery
client = bigquery.Client(project=project_id, credentials=credentials)
table_id = f"{project_id}.{dataset_id}.{table_id}"

In [116]:
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("link", "STRING"),
        bigquery.SchemaField("title", "STRING"),
        bigquery.SchemaField("date", "STRING"),
        bigquery.SchemaField("newspaper_section", "STRING"),
    ],
    write_disposition="WRITE_TRUNCATE",  # Modo de escrita: WRITE_TRUNCATE irá substituir a tabela existente
)


In [117]:
# Carrega os dados para o BigQuery
job = client.load_table_from_dataframe(df, table_id, job_config=job_config)

In [118]:
job.result()  # Aguarda a conclusão do job
print(f"Os dados foram enviados para a tabela: {table_id}")

Os dados foram enviados para a tabela: projeto-noticias-394817.noticias.folha
